In [1]:
import time
import numpy as np
import DRL.takeoff_dqn as tfdqn
import LLCsimple
import LLC 
import scaffold.fgdata as dfer
import scaffold.pidpilot as PID
import datetime
import fgmodule.fgenv as fgenv
import pandas as pd
import os
from scaffold.timer import gettime

'''
##########controlframe############


##########state_dict##############
{'aileron': 0.0, 'elevator': 0.0, 'rudder': -0.026, 'flaps': 0.0, 'throttle0': 0.6, 'throttle1': 0.6, 'vsi-fpm': 0.0, 'alt-ft': -372.808502, 'ai-pitch': 0.401045, 'ai-roll': 0.050598, 'ai-offset': 0.0, 'hi-heading': 80.568947, 'roll-deg': 0.050616, 'pitch-deg': 0.401055, 'heading-deg': 90.013458, 'airspeed-kt': 71.631187, 'speed-north-fps': -0.021637, 'speed-east-fps': 119.609383, 'speed-down-fps': -0.071344, 'uBody-fps': 119.606964, 'vBody-fps': -0.005778, 'wBody-fps': 0.765776, 'north-accel-fps_sec': 0.118887, 'east-accel-fps_sec': 5.870498, 'down-accel-fps_sec': -0.003219, 'x-accel-fps_sec': 6.095403, 'y-accel-fps_sec': -0.148636, 'z-accel-fps_sec': -32.113453, 'latitude': 21.325245, 'longitude': -157.93947, 'altitude': 22.297876, 'crashed': 0.0}
'''

LLC_FEATURE_BOUNDS = {
    'aileron': [-1, 1],  # 副翼 控制飞机翻滚 [-1,1] left/right
    'rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
    'pitch-deg': [-90., 90.],  # 飞机俯仰角
    'roll-deg': [-180., 180.],  # 飞机滚转角
    'heading-deg': [0., 360.],  # 飞机朝向
}
LLC_GOAL_BOUNDS = {
    'pitch-deg': [-90., 90.],  # 飞机俯仰角
    'roll-deg': [-180., 180.],  # 飞机滚转角
    'heading-deg': [0., 360.],  # 飞机朝向
}
goals = {
    'pitch-deg': 0.,  # 飞机俯仰角
    'roll-deg': 0.,  # 飞机滚转角
    'heading-deg': 90.,  # 飞机朝向
}

LLC_ACTION_BOUNDS = {
    'aileron': [-1, 1],  # 副翼 控制飞机翻滚 [-1,1] left/right
    'rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
}

def fgstart(fg2client_addr=("127.0.0.1", 5700), client2fg_addr=("127.0.0.1", 5701), telnet_addr=("127.0.0.1", 5555)):
    '''
    简化FG通信启动过程
    ---
    Inputs:
        - fg2client_addr
        - client2fg_addr
        - telnet_addr
    Outputs:
        - myfgenv:初始话完成的fgenv类

    '''
    myfgenv = fgenv.fgenv(telnet_addr, fg2client_addr, client2fg_addr)
    # initial_state = myfgenv.initial()
    myfgenv.initial()

    return myfgenv


MAX_EPISODE = 1000
MAX_EP_STEPS = 200

GAMMA = 0.9
LR_A = 0.001    # learning rate for actor
LR_C = 0.01     # learning rate for critic

##
epoch = 1000
step = 220

In [10]:
myfgenv = fgstart()

# bounds = {
#     'rudder': [-1, 1],  # 方向舵 控制飞机转弯（地面飞机方向控制） 0 /enter
# }

flylog saving path check pass!
send data to 127.0.0.1:5701 !
fgudp is ready!!!
Bind UDP on 127.0.0.1:5700 !
save log to data/flylog
auto start完成 18 : 1 : 28
fgudp stop begin! waiting for 2 thread quit
send data to 127.0.0.1:5701 stop!
UDP rece on 127.0.0.1:5700 stop!
reset完成 18 : 1 : 29
flylog saving path check pass!
send data to 127.0.0.1:5701 !
fgudp is ready!!!
Bind UDP on 127.0.0.1:5700 !
save log to data/flylog
auto start完成 18 : 2 : 24


In [2]:
myllc = LLCsimple.LLC(LLC_FEATURE_BOUNDS,LLC_GOAL_BOUNDS,LLC_ACTION_BOUNDS)

In [ ]:
state = myfgenv.replay("sky")

In [3]:
state = {'aileron': 0.0,
 'elevator': 0.0,
 'rudder': 0.0,
 'flaps': 0.0,
 'throttle0': 0.0,
 'throttle1': 0.0,
 'vsi-fpm': 0.0,
 'alt-ft': -370.319122,
 'ai-pitch': 0.534232,
 'ai-roll': 0.277171,
 'ai-offset': 0.0,
 'hi-heading': 83.792358,
 'roll-deg': 0.056131,
 'pitch-deg': 0.467942,
 'heading-deg': 90.00074,
 'airspeed-kt': 1.532065,
 'speed-north-fps': -0.000281,
 'speed-east-fps': -0.007067,
 'speed-down-fps': 0.000105,
 'uBody-fps': -0.007067,
 'vBody-fps': 0.000281,
 'wBody-fps': 4.7e-05,
 'north-accel-fps_sec': -1.4e-05,
 'east-accel-fps_sec': -0.009769,
 'down-accel-fps_sec': -0.000142,
 'x-accel-fps_sec': 0.252822,
 'y-accel-fps_sec': -0.031483,
 'z-accel-fps_sec': -32.151367,
 'latitude': 21.325247,
 'longitude': -157.943137,
 'altitude': 21.358186,
 'crashed': 0.0}

In [4]:
goal = goals

In [5]:
myllc.goals

{'pitch-deg': [-90.0, 90.0],
 'roll-deg': [-180.0, 180.0],
 'heading-deg': [0.0, 360.0]}

array([-0.30626116,  0.13050586, -0.09737893])

In [9]:
action ,_ = myllc.choose_action(state,goal)
print(LLCsimple.llc_reward(state , goal, old_action, action, 0))
print(action)
print(old_action)
old_action = action

-0.49682351424715865
[-0.5381245  -0.25378835]
[-0.64095265  0.4908171 ]


In [9]:
import importlib
importlib.reload(LLCsimple)

<module 'LLCsimple' from 'C:\\Users\\niebu\\Desktop\\FlightGear\\FGAutopilot\\FGAutopilot\\LLCsimple.py'>

In [6]:
action = np.array([0.,0.])
old_action = np.array([0.,0.])

In [ ]:
 for e in range(epoch):
        state = myfgenv.replay("sky")
        time.sleep(2)

        next_goal = goal

        ep_reward = 0

        for s in range(step):
        
            
            old_action = action 
            action,action_true = myllc.choose_action(state,goal)
            
            elevator = -0.01*(0.0-state['pitch-deg'])

            action_frame = dfer.action2frame((action_true[0],elevator,action_true[1],0.6,0.6))
            
            next_state, reward , done , info = myfgenv.step(action_frame) 
            
            r_ = LLCsimple.llc_reward(state , goal,old_action,action ,reward)
            
            next_goal = goals
            myllc.learn(state, goal, r_, action,next_state , next_goal)
            
            state = next_state
            goal = next_goal

            ep_reward += r_
            if done:
                print('Episode:', e, ' Reward: %i' %
                      int(ep_reward), 'Explore: %.2f' % myllc.var, )
                break
            
            if s == step-1:
                print('Episode:', e, ' Reward: %i' %
                      int(ep_reward), 'Explore: %.2f' % myllc.var, )
                # if ep_reward > -300:RENDER = True
                break
            time.sleep(0.1)

replay to sky 完成!time:18 : 3 : 6
auto start完成 18 : 3 : 9
Episode: 0  Reward: -319 Explore: 0.30
replay to sky 完成!time:18 : 3 : 35
auto start完成 18 : 3 : 38
Episode: 1  Reward: -298 Explore: 0.30
replay to sky 完成!time:18 : 4 : 5
auto start完成 18 : 4 : 8
Episode: 2  Reward: -317 Explore: 0.30
replay to sky 完成!time:18 : 4 : 35
auto start完成 18 : 4 : 38
Episode: 3  Reward: -311 Explore: 0.30
replay to sky 完成!time:18 : 5 : 5
auto start完成 18 : 5 : 8
Episode: 4  Reward: -303 Explore: 0.30
replay to sky 完成!time:18 : 5 : 35
auto start完成 18 : 5 : 38
Episode: 5  Reward: -299 Explore: 0.30
replay to sky 完成!time:18 : 6 : 5
auto start完成 18 : 6 : 8
Episode: 6  Reward: -308 Explore: 0.30
replay to sky 完成!time:18 : 6 : 35
auto start完成 18 : 6 : 38
Episode: 7  Reward: -315 Explore: 0.30
replay to sky 完成!time:18 : 7 : 5
auto start完成 18 : 7 : 8
Episode: 8  Reward: -319 Explore: 0.30
replay to sky 完成!time:18 : 7 : 35
auto start完成 18 : 7 : 38
Episode: 9  Reward: -296 Explore: 0.30
replay to sky 完成!time:18 : 8 :

auto start完成 18 : 43 : 34
Episode: 82  Reward: -298 Explore: 0.30
replay to sky 完成!time:18 : 44 : 0
auto start完成 18 : 44 : 3
Episode: 83  Reward: -298 Explore: 0.30
replay to sky 完成!time:18 : 44 : 30
auto start完成 18 : 44 : 33
Episode: 84  Reward: -308 Explore: 0.30
replay to sky 完成!time:18 : 45 : 0
auto start完成 18 : 45 : 3
Episode: 85  Reward: -291 Explore: 0.30
replay to sky 完成!time:18 : 45 : 29
auto start完成 18 : 45 : 32
Episode: 86  Reward: -304 Explore: 0.30
replay to sky 完成!time:18 : 45 : 59
auto start完成 18 : 46 : 2
Episode: 87  Reward: -293 Explore: 0.30
replay to sky 完成!time:18 : 46 : 28
auto start完成 18 : 46 : 31
Episode: 88  Reward: -305 Explore: 0.30
replay to sky 完成!time:18 : 46 : 58
auto start完成 18 : 47 : 1
Episode: 89  Reward: -289 Explore: 0.30
replay to sky 完成!time:18 : 47 : 27
auto start完成 18 : 47 : 31
Episode: 90  Reward: -300 Explore: 0.30
replay to sky 完成!time:18 : 47 : 57
auto start完成 18 : 48 : 0
Episode: 91  Reward: -303 Explore: 0.30
replay to sky 完成!time:18 : 48 : 

Episode: 163  Reward: -301 Explore: 0.30
replay to sky 完成!time:19 : 23 : 53
auto start完成 19 : 23 : 56
Episode: 164  Reward: -294 Explore: 0.30
replay to sky 完成!time:19 : 24 : 23
auto start完成 19 : 24 : 26
Episode: 165  Reward: -306 Explore: 0.30
replay to sky 完成!time:19 : 24 : 52
auto start完成 19 : 24 : 55
Episode: 166  Reward: -311 Explore: 0.30
replay to sky 完成!time:19 : 25 : 22
auto start完成 19 : 25 : 25
Episode: 167  Reward: -298 Explore: 0.30
replay to sky 完成!time:19 : 25 : 51
auto start完成 19 : 25 : 54
Episode: 168  Reward: -307 Explore: 0.30
replay to sky 完成!time:19 : 26 : 21
auto start完成 19 : 26 : 24
Episode: 169  Reward: -286 Explore: 0.30
replay to sky 完成!time:19 : 26 : 51
auto start完成 19 : 26 : 54
Episode: 170  Reward: -310 Explore: 0.30
replay to sky 完成!time:19 : 27 : 20
auto start完成 19 : 27 : 23
Episode: 171  Reward: -317 Explore: 0.30
replay to sky 完成!time:19 : 27 : 50
auto start完成 19 : 27 : 53
Episode: 172  Reward: -312 Explore: 0.30
replay to sky 完成!time:19 : 28 : 19
auto s

auto start完成 20 : 2 : 39
Episode: 244  Reward: -205 Explore: 0.25
replay to sky 完成!time:20 : 3 : 0
auto start完成 20 : 3 : 3
Episode: 245  Reward: -248 Explore: 0.25
replay to sky 完成!time:20 : 3 : 26
auto start完成 20 : 3 : 29
Episode: 246  Reward: -249 Explore: 0.25
replay to sky 完成!time:20 : 3 : 53
auto start完成 20 : 3 : 56
Episode: 247  Reward: -249 Explore: 0.25
replay to sky 完成!time:20 : 4 : 19
auto start完成 20 : 4 : 22
Episode: 248  Reward: -302 Explore: 0.24
replay to sky 完成!time:20 : 4 : 48
auto start完成 20 : 4 : 51
Episode: 249  Reward: -257 Explore: 0.24
replay to sky 完成!time:20 : 5 : 17
auto start完成 20 : 5 : 20
Episode: 250  Reward: -259 Explore: 0.24
replay to sky 完成!time:20 : 5 : 47
auto start完成 20 : 5 : 50
Episode: 251  Reward: -276 Explore: 0.23
replay to sky 完成!time:20 : 6 : 17
auto start完成 20 : 6 : 20
Episode: 252  Reward: -267 Explore: 0.23
replay to sky 完成!time:20 : 6 : 46
auto start完成 20 : 6 : 49
Episode: 253  Reward: -266 Explore: 0.23
replay to sky 完成!time:20 : 7 : 16
au

auto start完成 20 : 42 : 21
Episode: 325  Reward: -263 Explore: 0.10
replay to sky 完成!time:20 : 42 : 47
auto start完成 20 : 42 : 50
Episode: 326  Reward: -262 Explore: 0.10
replay to sky 完成!time:20 : 43 : 17
auto start完成 20 : 43 : 20
Episode: 327  Reward: -252 Explore: 0.10
replay to sky 完成!time:20 : 43 : 47
auto start完成 20 : 43 : 50
Episode: 328  Reward: -266 Explore: 0.10
replay to sky 完成!time:20 : 44 : 16
auto start完成 20 : 44 : 19
Episode: 329  Reward: -258 Explore: 0.10
replay to sky 完成!time:20 : 44 : 46
auto start完成 20 : 44 : 49
Episode: 330  Reward: -217 Explore: 0.10
replay to sky 完成!time:20 : 45 : 16
auto start完成 20 : 45 : 19
Episode: 331  Reward: -264 Explore: 0.10
replay to sky 完成!time:20 : 45 : 45
auto start完成 20 : 45 : 48
Episode: 332  Reward: -257 Explore: 0.10
replay to sky 完成!time:20 : 46 : 15
auto start完成 20 : 46 : 18
Episode: 333  Reward: -260 Explore: 0.10
replay to sky 完成!time:20 : 46 : 45
auto start完成 20 : 46 : 48
Episode: 334  Reward: -251 Explore: 0.09
replay to sky 完

auto start完成 21 : 21 : 17
Episode: 406  Reward: -245 Explore: 0.04
replay to sky 完成!time:21 : 21 : 43
auto start完成 21 : 21 : 46
Episode: 407  Reward: -239 Explore: 0.04
replay to sky 完成!time:21 : 22 : 13
auto start完成 21 : 22 : 16
Episode: 408  Reward: -250 Explore: 0.04
replay to sky 完成!time:21 : 22 : 42
auto start完成 21 : 22 : 45
Episode: 409  Reward: -240 Explore: 0.04
replay to sky 完成!time:21 : 23 : 12
auto start完成 21 : 23 : 15


In [16]:
myllc.save("modelckpt/LLCsimple_mlj_d5g3a2_pid_200/LLCsimple_d5g3a2_200.ckpt")

In [38]:
myllc.ddpg.pointer

25081

In [37]:
t1= time.clock()
myllc.learn(state, goal, r_, action,next_state , next_goal)
print(time.clock()-t1)

0.0027973283603017762
